In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix, classification_report

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'In_Class_Assignments/Iris.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
iris = pd.read_csv(file_content_stream)
iris.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
## Changing labels to numbers
iris['Species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1, 
                                np.where(iris['Species'] == 'Iris-versicolor', 2, 3))
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Species_numb
0,1,5.1,3.5,1.4,0.2,Iris-setosa,3
1,2,4.9,3.0,1.4,0.2,Iris-setosa,3
2,3,4.7,3.2,1.3,0.2,Iris-setosa,3
3,4,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5,5.0,3.6,1.4,0.2,Iris-setosa,3


In [3]:
## Defining inputs and target
X = iris.drop(columns = ['Id', 'Species', 'Species_numb'], axis = 1)
Y = iris['Species_numb']

## Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [4]:
## Scaling the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# One-vs-One (Logistic Regression)

In [5]:
## Building the model 
one_vs_one_LR = OneVsOneClassifier(estimator = LogisticRegression()).fit(X_train, Y_train)

## Predicting on the test 
one_vs_one_LR_pred = one_vs_one_LR.predict(X_test)

print(confusion_matrix(Y_test, one_vs_one_LR_pred))
print(classification_report(Y_test, one_vs_one_LR_pred))

[[ 9  1  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



# One-vs-One (Decision Tree)

In [6]:
## Building the model 
one_vs_one_tree = OneVsOneClassifier(estimator = DecisionTreeClassifier(max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test 
one_vs_one_tree_pred = one_vs_one_tree.predict(X_test)

print(confusion_matrix(Y_test, one_vs_one_tree_pred))
print(classification_report(Y_test, one_vs_one_tree_pred))

[[ 9  1  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
# From my results, both models have the same performance.